# Shortwave Radiation Transfer Example

This notebook demonstrates how to compute shortwave radiation fluxes using the `pyrte_rrtmgp` library. Specifically:

- It downloads the required data
- Computes the gas optics using a two-stream approximation
- Solves the radiation transfer equations
- Validates the results against reference datasets

See the [documentation](https://pyrte-rrtmgp.readthedocs.io/en/latest/) for more information.

In [1]:
import os

import numpy as np
import xarray as xr

## Importing pyrte_rrtmgp Modules

We import the necessary modules from the `pyrte_rrtmgp` package which provide tools for gas optics and solving the radiation transfer equations.

In [2]:
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rte_solver import rte_solve
from pyrte_rrtmgp.data_types import RFMIPExampleFiles
from pyrte_rrtmgp.utils import load_rrtmgp_file

ERROR_TOLERANCE = 1e-7

## Loading Gas Optics Data and Atmosphere Dataset

Load the shortwave gas optics file and the associated atmosphere dataset that will be used for the simulation.

In [ ]:
gas_optics_sw = load_gas_optics(gas_optics_file=GasOpticsFiles.SW_G224)

atmosphere = load_rrtmgp_file(RFMIPExampleFiles.RFMIP).chunk({"expt": 3})
atmosphere

## Computing Gas Optics and Solving Radiation Transfer Equations

Compute the shortwave gas optics using a two-stream approximation and then solve the radiation transfer equations to obtain the fluxes.

In [ ]:
gas_optics_sw.gas_optics.compute(atmosphere, problem_type="two-stream")

fluxes = rte_solve(atmosphere, add_to_input=False)
fluxes

## Loading Reference Data and Validating the Results

Load the reference datasets for upward and downward shortwave fluxes and validate that the computed fluxes match the reference data within the specified tolerance.

In [ ]:
rsu = load_rrtmgp_file(RFMIPExampleFiles.REFERENCE_RSU)
rsd = load_rrtmgp_file(RFMIPExampleFiles.REFERENCE_RSD)

assert np.isclose(
    fluxes["sw_flux_up"], rsu["rsu"], atol=ERROR_TOLERANCE
).all(), "Shortwave flux up mismatch"
assert np.isclose(
    fluxes["sw_flux_down"], rsd["rsd"], atol=ERROR_TOLERANCE
).all(), "Shortwave flux down mismatch"

print("Shortwave radiation transfer calculations validated successfully!")